## Práctica Sympy 2

En base a la simulacion, generar una simulacion lo mas apegado a la realidad del Covid-19, para ello obtener informacion del numero de consultorios y camas disponibles en un hospital del Ecuador. Parametrizar la simulacion para ingresar el numero de pacientes, el numero de consultorios y el numero de camas y algun otro parametro que considere oportuno.

Finalmente generar las siguientes metricas :

Tiempo de espera promedio para cada sala.
Tiempo promedio total que requiere un paciente para ser atendido desde cero.
Cuántos pacientes se atendieron.
Cuántos pacientes se quedaron sin atender y fallecieron.


### Hospital General Alfredo Noboa Montenegro

In [ ]:
import random
import matplotlib.pyplot as pp
import numpy as np
import simpy

%matplotlib inline


numeroHospitales = 1
numeroCamas= 78
numeroConsultorios = 4
contagiados = 59
crecimiento = 11
tiempoConsulta = 0.015 
internado = 10
simulacion = 10
recuperados={}
muertos={}
tConsultas=[]
TiempoEsperaConsultorio=[]
pacienteMuertosSinAntencionM=[]
pacientesAtendidos=[]



class Hospital(object):
    #constructor
    def __init__(self, env, num_cama,num_consultorios, name):
            self.env = env
            self.num_cama = num_cama
            self.num_consultorios=num_consultorios
            self.camas = simpy.Resource(env, num_cama)
            self.consultorios=simpy.Resource(env, num_consultorios)
            self.name = name
    def ingresarConsultorio(self,paciente):
        consulta=random.uniform(tiempoConsulta-0.0005, tiempoConsulta+0.0005)
        yield self.env.timeout(consulta)
        print("El paciente ", paciente, " espero", int(consulta*1440), "minutos para se atendido")
        tConsultas.append(consulta)
        
    def PacienteCama(self, paciente):
        yield self.env.timeout(random.randint(internado-2, internado+2))
        print("El paciente ", paciente, " fue internado el día ", int(self.env.now))

def llegada_paciente(env, hospital, paciente):
    arrive = env.now
    with hospital.consultorios.request() as consultorio:
        esperandoConsulta=random.randint(1,3)
        TiempoEsperaConsultorio.append(esperandoConsulta)
        reque2 = yield consultorio | env.timeout(esperandoConsulta)
        espera2 = env.now - arrive
        if consultorio in reque2:
            print("Al paciente: ", paciente, " se le asigna una cama")
            yield env.process(hospital.ingresarConsultorio(paciente))
            estado = random.randint(1,100)
            pacientesAtendidos.append(1)
            if(estado < 40):
                with hospital.camas.request() as cama:
                    arrive2 = env.now
                    dias_esperando = random.randint(1,5) 
                    requerimiento = yield cama | env.timeout(dias_esperando) # tiempo de espera
                    espera = env.now - arrive2
                    if cama in requerimiento:
                        print("Al paciente: ", paciente, " se le asigna una cama")
                        yield env.process(hospital.PacienteCama(paciente))
                        estado = random.randint(1,100)
                        if (estado < 8) :
                            muertos[env.now] = muertos[env.now] + 1 if env.now in muertos else  1
                        else:
                            recuperados[env.now] = recuperados[env.now] + 1 if env.now in recuperados else 1
                    else:
                        print("El paciente " , paciente, " espero ", int(espera) , " dias y fallece por la falta de camas")
                        muertos[env.now] = muertos[env.now] + 1 if env.now in muertos else  1
            else :
                print("El paciente ", paciente, " no tiene COVID-19" )
        else:
            print("El paciente " , paciente, "espero ", int(espera2) ," dias y fallece porque no fue atendido")
            pacienteMuertosSinAntencionM.append(1)
            muertos[env.now] = muertos[env.now] + 1 if env.now in muertos else  1
def ejecutar(env, tasacrecimiento, infectados):
    hospitalMoreno = Hospital(env, numeroCamas,numeroConsultorios, "Moreno")
    for i in range(infectados):
        asignar_hospital(env, hospitalMoreno, i)
    paciente = infectados
    while True:
        yield env.timeout(1)
        for i in range(tasacrecimiento):
            paciente += 1
            asignar_hospital(env, hospitalMoreno, paciente)
                
def asignar_hospital(env, hospitalMoreno, paciente):
    hosp_esc = 1
    if (hosp_esc == 1):
        print("Llega un paciente nuevo: ", paciente, "al Hospital General Alfredo Noboa")
        env.process(llegada_paciente(env, hospitalMoreno, paciente))

print("Simulacion COVID 19")
env=simpy.Environment()
env.process(ejecutar(env,crecimiento, contagiados))
env.run(until=simulacion)

print("Resultados pacientes :")
print("Recuperados: ")
print(recuperados)
print("Fallecidos: ")
print(muertos)

datos=sorted(recuperados.items())
x, y =zip(*datos) 
pp.plot(x,y,linewidth=2,color='y') 
pp.scatter(x,y,color='g')
pp.title("Dias  / Personas Recuperadas")
pp.grid(True)
pp.show()

if (muertos):
    datos=sorted(muertos.items())
    x, y =zip(*datos)
    pp.plot(x,y,linewidth=2,color='y')
    pp.scatter(x,y,color='g')
    pp.title("Fallecidos  / dias hospitalaiados")
    pp.grid(True)
    pp.show() 
print(tConsultas)
print(TiempoEsperaConsultorio)